In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Import the necessary packages

import pandas as pd
import numpy as np
import os
import PIL
import seaborn as sns
import pickle
from PIL import *
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler
from IPython.display import display
from tensorflow.keras import Input
from tensorflow.python.keras import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, optimizers
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import *
from tensorflow.keras import backend as K
from keras import optimizers
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Reading Data

In [ ]:
keyfacial_df = pd.read_csv('../input/facial-keypoints-detection/training.zip', compression='zip', header=0, sep=',', quotechar='"')
test_data = pd.read_csv('../input/facial-keypoints-detection/test.zip', compression='zip', header=0, sep=',', quotechar='"')
IdLookupTable = pd.read_csv('../input/facial-keypoints-detection/IdLookupTable.csv',header=0, sep=',', quotechar='"')
SampleSubmission = pd.read_csv('../input/facial-keypoints-detection/SampleSubmission.csv',header=0, sep=',', quotechar='"')

In [ ]:
keyfacial_df.head()

In [ ]:
keyfacial_df.info()

# Missing values

In [ ]:
keyfacial_df.isnull().sum()

In [ ]:
keyfacial_df.fillna(method = 'ffill',inplace = True)

# Reshape Image

In [ ]:
keyfacial_df['Image'].shape

In [ ]:
keyfacial_df['Image']=keyfacial_df['Image'].apply(lambda x:np.fromstring(x,dtype=int,sep=' ').reshape(96,96))

In [ ]:
keyfacial_df['Image'][0].shape

In [ ]:
keyfacial_df.describe()

# Image visulaization

In [ ]:
# Plot a random image from the dataset along with facial keypoints. 
# Image data is obtained from df['Image'] and plotted using plt.imshow
# 15 x and y coordinates for the corresponding image 
# since x-coordinates are in even columns like 0,2,4,.. and y-coordinates are in odd columns like 1,3,5,..
# we access their value using .loc command, which get the values for coordinates of the image based on the column it is refering to.

i = np.random.randint(1, len(keyfacial_df))
plt.imshow(keyfacial_df['Image'][i], cmap = 'gray')
for j in range(1, 31, 2):
        plt.plot(keyfacial_df.loc[i][j-1], keyfacial_df.loc[i][j], 'rx')


In [ ]:
import random
# Let's view more images in a grid format
fig = plt.figure(figsize=(20, 20))

for i in range(16):
    k=random.randint(1,len(keyfacial_df))
    ax = fig.add_subplot(4, 4, i + 1)
    image = plt.imshow(keyfacial_df['Image'][k],cmap = 'gray')
    for j in range(1,31,2):
        plt.plot(keyfacial_df.loc[k][j-1], keyfacial_df.loc[k][j], 'rx')
    

# Image Augmentation

In [ ]:
#Creating a new copy of the dataframe
import copy
keyfacial_df_copy=copy.copy(keyfacial_df)

In [ ]:
columns =keyfacial_df_copy.columns[:-1]
columns

> * Horizontal flip

In [ ]:
#horizontal-flip
keyfacial_df_copy['Image']=keyfacial_df_copy['Image'].apply(lambda x:np.flip(x,axis=1))
#As Flipping horizontally , y coordiantes will be same 
# only x cordinate values will change ,So Subtract our x-coordinate  values from width of the image 96
for i in range(len(columns)):
    if i%2 == 0:
         keyfacial_df_copy[columns[i]] = keyfacial_df_copy[columns[i]].apply(lambda x:96.-float(x))

In [ ]:

# Original image
plt.imshow(keyfacial_df['Image'][1], cmap = 'gray')
for j in range(1, 31, 2):
        plt.plot(keyfacial_df.loc[1][j-1], keyfacial_df.loc[1][j], 'rx')


In [ ]:

# horizontal flip  image
plt.imshow(keyfacial_df_copy['Image'][1], cmap = 'gray')
for j in range(1, 31, 2):
        plt.plot(keyfacial_df_copy.loc[1][j-1], keyfacial_df_copy.loc[1][j], 'rx')


In [ ]:
# Concatenating the original dataframe with the augmented dataframe
augmented_df = np.concatenate((keyfacial_df, keyfacial_df_copy))

In [ ]:
augmented_df.shape

> * Brightness

In [ ]:
# Randomingly increasing the brightness of the images
# We multiply pixel values by random values between 1.5 and 2 to increase the brightness of the image
# we clip the value between 0 and 255

import random

keyfacial_df_copy = copy.copy(keyfacial_df)
keyfacial_df_copy['Image'] = keyfacial_df_copy['Image'].apply(lambda x:np.clip(random.uniform(1.5, 2)* x, 0.0, 255.0))
augmented_df = np.concatenate((augmented_df, keyfacial_df_copy))
augmented_df.shape

In [ ]:
#Image with increased brightness

plt.imshow(keyfacial_df_copy['Image'][1], cmap='gray')
for j in range(1, 31, 2):
        plt.plot(keyfacial_df_copy.loc[1][j-1], keyfacial_df_copy.loc[1][j], 'rx')

> * Vertical Flip

In [ ]:
#Vertical flip 
#x-coordiantes will not change
#y-coordinates will change 
#horizontal-flip
keyfacial_df_copy['Image']=keyfacial_df_copy['Image'].apply(lambda x:np.flip(x,axis=0))
for i in range(len(columns)):
    if i%2 == 1:
         keyfacial_df_copy[columns[i]] = keyfacial_df_copy[columns[i]].apply(lambda x:96.-float(x))

In [ ]:
plt.imshow(keyfacial_df_copy['Image'][1], cmap='gray')
for j in range(1, 31, 2):
        plt.plot(keyfacial_df_copy.loc[1][j-1], keyfacial_df_copy.loc[1][j], 'rx')

# Data Normalization and Scaling

In [ ]:
#values of Image
img=augmented_df[:,30]

#Normalize the image
img=img/255.

# empty array to feed the model of shape(96,96,1)
X= np.empty((len(img),96,96,1))

#expanding dimensions to (96,96,1)
for i in range(len(img)):
    X[i,]=np.expand_dims(img[i],axis=2)

#Converting array type to float
X=np.asarray(X).astype(np.float32)


In [ ]:
X.shape

In [ ]:
y=augmented_df[:,:30]
y=np.asarray(y).astype(np.float32)
y.shape

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

# Residual Neural Network

In [ ]:
def res_block(X, filter, stage):
    
    

  # Convolutional_block
    X_copy = X

    f1 , f2, f3 = filter

  # Main Path
    X = Conv2D(f1, (1,1),strides = (1,1), name ='res_'+str(stage)+'_conv_a', kernel_initializer= glorot_uniform(seed = 0))(X)
    X = MaxPool2D((2,2))(X)
    X = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_conv_a')(X)
    X = Activation('relu')(X) 

    X = Conv2D(f2, kernel_size = (3,3), strides =(1,1), padding = 'same', name ='res_'+str(stage)+'_conv_b', kernel_initializer= glorot_uniform(seed = 0))(X)
    X = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_conv_b')(X)
    X = Activation('relu')(X) 

    X = Conv2D(f3, kernel_size = (1,1), strides =(1,1),name ='res_'+str(stage)+'_conv_c', kernel_initializer= glorot_uniform(seed = 0))(X)
    X = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_conv_c')(X)

  # Short path
    X_copy = Conv2D(f3, kernel_size = (1,1), strides =(1,1),name ='res_'+str(stage)+'_conv_copy', kernel_initializer= glorot_uniform(seed = 0))(X_copy)
    X_copy = MaxPool2D((2,2))(X_copy)
    X_copy = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_conv_copy')(X_copy)

  # ADD
    X = Add()([X,X_copy])
    X = Activation('relu')(X)

  # Identity Block 1
    X_copy = X


  # Main Path
    X = Conv2D(f1, (1,1),strides = (1,1), name ='res_'+str(stage)+'_identity_1_a', kernel_initializer= glorot_uniform(seed = 0))(X)
    X = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_identity_1_a')(X)
    X = Activation('relu')(X) 

    X = Conv2D(f2, kernel_size = (3,3), strides =(1,1), padding = 'same', name ='res_'+str(stage)+'_identity_1_b', kernel_initializer= glorot_uniform(seed = 0))(X)
    X = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_identity_1_b')(X)
    X = Activation('relu')(X) 

    X = Conv2D(f3, kernel_size = (1,1), strides =(1,1),name ='res_'+str(stage)+'_identity_1_c', kernel_initializer= glorot_uniform(seed = 0))(X)
    X = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_identity_1_c')(X)

  # ADD
    X = Add()([X,X_copy])
    X = Activation('relu')(X)

  # Identity Block 2
    X_copy = X


  # Main Path
    X = Conv2D(f1, (1,1),strides = (1,1), name ='res_'+str(stage)+'_identity_2_a', kernel_initializer= glorot_uniform(seed = 0))(X)
    X = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_identity_2_a')(X)
    X = Activation('relu')(X) 

    X = Conv2D(f2, kernel_size = (3,3), strides =(1,1), padding = 'same', name ='res_'+str(stage)+'_identity_2_b', kernel_initializer= glorot_uniform(seed = 0))(X)
    X = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_identity_2_b')(X)
    X = Activation('relu')(X) 

    X = Conv2D(f3, kernel_size = (1,1), strides =(1,1),name ='res_'+str(stage)+'_identity_2_c', kernel_initializer= glorot_uniform(seed = 0))(X)
    X = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_identity_2_c')(X)

  # ADD
    X = Add()([X,X_copy])
    X = Activation('relu')(X)

    return X

In [ ]:
input_shape = (96, 96, 1)

# Input tensor shape
X_input = Input(input_shape)

# Zero-padding
X = ZeroPadding2D((3,3))(X_input)

# 1 - stage
X = Conv2D(64, (7,7), strides= (2,2), name = 'conv1', kernel_initializer= glorot_uniform(seed = 0))(X)
X = BatchNormalization(axis =3, name = 'bn_conv1')(X)
X = Activation('relu')(X)
X = MaxPooling2D((3,3), strides= (2,2))(X)

# 2 - stage
X = res_block(X, filter= [64,64,256], stage= 2)

# 3 - stage
X = res_block(X, filter= [128,128,512], stage= 3)


# Average Pooling
X = AveragePooling2D((2,2), name = 'Averagea_Pooling')(X)

# Final layer
X = Flatten()(X)
X = Dense(4096, activation = 'relu')(X)
X = Dropout(0.2)(X)
X = Dense(2048, activation = 'relu')(X)
X = Dropout(0.1)(X)
X = Dense(30, activation = 'relu')(X)


model_1_facialKeyPoints = Model( inputs= X_input, outputs = X)
model_1_facialKeyPoints.summary()

# Compile and Traning

In [ ]:
adam = tf.keras.optimizers.Adam(learning_rate = 0.0001, beta_1 = 0.9, beta_2 = 0.999, amsgrad = False)
model_1_facialKeyPoints.compile(loss = "mean_squared_error", optimizer = adam , metrics = ['accuracy'])

In [ ]:
# save the best model with least validation loss
checkpointer = ModelCheckpoint(filepath = "FacialKeyPoints_weights.hdf5", verbose = 1, save_best_only = True)

In [ ]:
history = model_1_facialKeyPoints.fit(X_train, y_train, batch_size = 32, epochs = 100, validation_data=(X_test,y_test), callbacks=[checkpointer])